In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
from keras.datasets import fashion_mnist

In [ ]:
class PreProc:
    def __init__(self):
        (self.trainx,self.trainy),(self.testx, self.testy) = fashion_mnist.load_data()
    def visualize(self,n):
        for i in range(n):
            plt.subplot(330+1+i) # ask someone why??
            plt.imshow(self.trainx[i], cmap = plt.get_cmap('gray'))
        plt.show()
    def flatten(self):
        trainx_flattened = self.trainx
        testx_flattened = self.testx
        trainx_flattened.shape = (60000,784)
        testx_flattened.shape = (10000,784)
        return trainx_flattened,testx_flattened
    def getLabels(self):
        return self.trainy, self.testy
    def getInputSize(self):
        return len(self.trainx[0])

In [ ]:
data.getInputSize()

(784,)

In [ ]:
class Functions:
    @staticmethod
    def sigmoid(input):
        return  1/(1+np.exp(-input))
    @staticmethod
    def softmax(input):
        return np.exp(input)/(np.sum(np.exp(input)))
    @staticmethod
    def onehot(input):
        result = np.zeros(10)
        result[input] = 1
        return result
    @staticmethod
    def crossEntropyLoss(y,yHat):
        result = np.dot(y,np.log(yHat))
        return result
    @staticmethod
    def derivative_sigmoid(input):
        return ((np.exp(-input))/(1+np.exp(-input)**2))

In [47]:
class Algorithms:
    @staticmethod
    def ForwardProp(net, activate, output, inputLayer):
        L = len(net)-1
        a = []
        weights = net[0][:,:len(net[0][0])-1]
        bias = net[0][:,len(net[0][0])-1]
        temp = np.matmul(weights,inputLayer)+bias
        temp = temp/np.linalg.norm(temp)
        a.append(temp)
        h = []
        h.append(activate(a[0]))
        for k in range(1,L-1):
            weights = net[k][:,:len(net[k][0])-1]
            bias = net[k][:,len(net[k][0])-1]
            temp = np.matmul(weights,h[k-1])+bias
            temp = temp/np.linalg.norm(temp)
            a.append(bias + np.matmul(weights,h[k-1]))
            h.append(activate(a[k]))
        weights = net[L][:,:len(net[L][0])-1]
        bias = net[L][:,len(net[L][0])-1]
        temp = np.matmul(weights,h[0])+bias
        temp = temp/np.linalg.norm(temp)
        a.append(temp)
        h.append(output(a[L]))
        return a,h
    @staticmethod
    def BackProp(net, a, h, dataPoint):
        L = len(net)-1
        gradaL = -(Functions.onehot(train_y[1])-h[len(h)-1])
        gradient = np.zeros_like(net)
        for k in range(L,0,-1):
            gradW = np.outer(gradaL,h[k-1].T)
            gradB = gradaL
            gradB.resize((len(gradB),1))
            np.append(gradW,gradB,axis=1)
            gradient[k] = gradW

            gradhL_1 = np.matmul(np.transpose(net[k][:,len(net[k])-1]),a[k])
            gradaL_1 = np.multiply(gradhL_1, Functions.derivative_sigmoid(a[k-1]))
            print(net[k][:,:len(net[k][0])-1].shape)
            gradaL = gradaL_1
        gradW = np.outer(gradaL,dataPoint.T)
        gradB = gradaL
        gradB.resize((len(gradB),1))
        np.append(gradW,gradB,axis=1)
        gradient[0] = gradW
        return gradient

In [41]:
#The class of FeedForwardNeuralNetwor

class FFNet:
    #constructor
    def __init__(self,number_of_hidden_layers, number_of_inputs, number_of_outputs):
        self.number_of_inputs = number_of_inputs
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_outputs = number_of_outputs
        self.input = [0 for i in range(number_of_inputs)]
        self.output = [0 for i in range(10)]
        self.hidden = []
        #self.hidden.append(np.random.random((number_of_inputs+1)))
    
    #Method for creating layers
    def addHiddenLayer(self,number_of_neurons):
        if(len(self.hidden) == 0):
            temp_weights = np.random.random((number_of_neurons, self.number_of_inputs+1)) #The +1 is for biases.
        else:
            prev_neurons = len(self.hidden[len(self.hidden) - 1])
            temp_weights = np.random.random((number_of_neurons, prev_neurons + 1)) # The +1 is for biases.
        temp_weights = temp_weights/np.linalg.norm(temp_weights)
        self.hidden.append(temp_weights)
    
    def addOutputLayer(self, number_of_outputs):
        if(len(self.hidden) == 0):
            #print("number of inputs: "+str(self.number_of_inputs))
            temp_weights = np.random.random((number_of_outputs, self.number_of_inputs)) #Bias not needed for the output layer.
        else:
            prev_neurons = len(self.hidden[len(self.hidden) - 1])
            temp_weights = np.random.random((number_of_outputs, prev_neurons + 1)) #Bias not needed for the output layer.
        temp_weights = temp_weights/np.linalg.norm(temp_weights)
        self.hidden.append(temp_weights)

    def solidify(self):
        self.network = np.array(self.hidden)

    def getNetwork(self):
        return self.network
    
    def ForwardProp(self, activate, output, inputLayer):
        net = self.network
        L = len(self.network)-1
        a = []
        weights = net[0][:,:len(net[0][0])-1]
        bias = net[0][:,len(net[0][0])-1]
        temp = np.matmul(weights,inputLayer)+bias
        temp = temp/np.linalg.norm(temp)
        a.append(temp)
        h = []
        h.append(activate(a[0]))
        for k in range(1,L-1):
            weights = net[k][:,:len(net[k][0])-1]
            bias = net[k][:,len(net[k][0])-1]
            temp = np.matmul(weights,h[k-1])+bias
            temp = temp/np.linalg.norm(temp)
            a.append(bias + np.matmul(weights,h[k-1]))
            h.append(activate(a[k]))
        weights = net[L][:,:len(net[L][0])-1]
        bias = net[L][:,len(net[L][0])-1]
        temp = np.matmul(weights,h[0])+bias
        temp = temp/np.linalg.norm(temp)
        a.append(temp)
        h.append(output(a[L]))
        self.historyA = a
        self.historyh = h
        return a,h
    
    def loss(self, lossFunction, Y):
        self.predY = self.historyA[(len(self.historyA)-1)]
        lossFunction(Y,self.predY)

    def BackProp(self, a, h, dataPoint):
        net = self.network
        L = len(net)-1
        gradaL = -(Functions.onehot(train_y[1])-h[len(h)-1])
        gradient = np.zeros_like(net)
        for k in range(L,0,-1):
            gradW = np.outer(gradaL,h[k-1].T)
            gradB = gradaL
            gradB.resize((len(gradB),1))
            np.append(gradW,gradB,axis=1)
            gradient[k] = gradW

            gradhL_1 = np.matmul(np.transpose(net[k][:,len(net[k])-1]),a[k])
            gradaL_1 = np.multiply(gradhL_1, Functions.derivative_sigmoid(a[k-1]))
            print(net[k][:,:len(net[k][0])-1].shape)
            gradaL = gradaL_1
        gradW = np.outer(gradaL,dataPoint.T)
        print("gradaL: ", end = " ")
        print(gradaL)
        print("gradW: ", end = " ")
        print(gradW)
        gradB = gradaL
        gradB.resize((len(gradB),1))
        np.append(gradW,gradB,axis=1)
        gradient[0] = gradW
        return gradient

In [ ]:
for i in range(10,0,-1):
    print(i, end = " ")

10 9 8 7 6 5 4 3 2 1 

In [42]:
if __name__ == '__main__':
    data = PreProc()
    #data.visualize(5)
    train_x, test_x = data.flatten()
    train_y, test_y = data.getLabels()
    neuralNet = FFNet(0,data.getInputSize(), 10)
    neuralNet.addHiddenLayer(20)
    neuralNet.addOutputLayer(10)
    neuralNet.solidify()
    net = neuralNet.getNetwork()
    a,h = neuralNet.ForwardProp(Functions.sigmoid, Functions.softmax, train_x[1])
    gradient = neuralNet.BackProp(a,h, train_x[0])

(10, 20)
gradaL:  [0.0920129  0.09167097 0.09201342 0.09192534 0.09223669 0.0919
 0.09186913 0.09184943 0.09193042 0.09194273 0.09194843 0.09179016
 0.09181748 0.09185127 0.09205699 0.09172587 0.09193367 0.09194057
 0.09189407 0.0919297 ]
gradW:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


<ipython-input-41-28e6341a5206>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.network = np.array(self.hidden)


In [ ]:
for i in gradient[0]:
    print(i, end = " ")

In [ ]:
gradient[1] = gradW

In [ ]:
gradaL = -(Functions.onehot(train_y[1])-h[len(h)-1])

In [ ]:
gradhL_1 = np.matmul(np.transpose(net[(len(net)-1)]),aL)

In [ ]:
gradaL_1 = np.multiply(net[len(net)-1][:,:len(net[len(net)-1][0])-1], Functions.derivative_sigmoid(a[len(net)-2]))

In [ ]:
gradW = np.outer(gradaL,h[len(net)-2].T)
gradB = gradaL


In [ ]:
gradB.resize((len(gradB),1))

In [ ]:
gradB.shape

(10, 1)

In [ ]:
np.append(gradW,gradB.resize((10,1)),axis=1)

array([[-0.49974422, -0.49987732, -0.50045317, -0.49836884, -0.50235192,
        -0.50235109, -0.49928474, -0.49920566, -0.49775972, -0.49974985,
        -0.49777699, -0.50029926, -0.49888012, -0.50086685, -0.50025429,
        -0.49987588, -0.49888532, -0.49982542, -0.50135421, -0.50049566,
        -0.89963249],
       [ 0.05646266,  0.0564777 ,  0.05654276,  0.05630726,  0.05675728,
         0.05675719,  0.05641074,  0.05640181,  0.05623844,  0.05646329,
         0.05624039,  0.05652537,  0.05636503,  0.0565895 ,  0.05652029,
         0.05647753,  0.05636562,  0.05647183,  0.05664456,  0.05654756,
         0.10164328],
       [ 0.05479879,  0.05481338,  0.05487653,  0.05464797,  0.05508473,
         0.05508464,  0.05474841,  0.05473974,  0.05458118,  0.05479941,
         0.05458308,  0.05485965,  0.05470404,  0.05492189,  0.05485472,
         0.05481323,  0.05470461,  0.05480769,  0.05497533,  0.05488119,
         0.09864801],
       [ 0.05828847,  0.058304  ,  0.05837116,  0.05812805

In [ ]:
gradaL.shape

(10,)

In [ ]:
a[len(net)-2].shape

(20,)

In [ ]:
weights = net[0][:,:len(net[0][0])-1]
bias = net[0][:,len(net[0][0])-1]
temp = np.matmul(weights,train_x[0])+bias
temp = temp/np.linalg.norm(temp)
a = []
a.append(temp)
h = []
h.append(Functions.sigmoid(a[0]))

In [ ]:
weights = net[L][:,:len(net[L][0])-1]
bias = net[L][:,len(net[L][0])-1]
temp = np.matmul(weights,h[0])+bias
temp = temp/np.linalg.norm(temp)

In [ ]:
L = len(net)-1

In [ ]:
weights.shape

(10, 20)

In [ ]:
#The class of FeedForwardNeuralNetwor

class FFNet:
    #constructor
    hidden = []
    input = []
    output = []
    def __init__(self,number_of_hidden_layers, number_of_inputs, number_of_outputs):
        self.number_of_inputs = number_of_inputs
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_outputs = number_of_outputs
        #At the same time, the layers input layers mus also be initialized.

        input = [0 for i in range(number_of_inputs)]
        output = [0 for i in range(number_of_outputs)]
        hidden = [[]]

        #input and output layers are nothing but simple lists
    
    #Method for creating layers
    def add_hidden_layer(number_of_neurons):
        temp_weights = [0 for i in range(number_of_neurons+1)] #The +1 is for bias values
        hidden.append(temp_weights)
    
    def backward_propagate(a,h, pred_y):
        delthet[L] = -(exp(y) - pred_y) #with respect to output layer
        for k in range(0,L-1,-1):
            delthetw = np.matmul(delthet[k], h[k-1].T)
            delthetb = delthet[k]
            deltheth = np.matmul(weights[k].T, delthet[k])
            delthet[k-1] = hadamard(deltheth, preac(a)) 

    def forward_propagate():
        #here, we are calculating the preactivations and activations.
        #we then store them in an array and return it.
        
        for k in range(number_of_levels-1):
            a[k] = biases[k] + np.matmul(weights[k], h[k-1])
            h[k] = g(a[k])
        a[number_of_levels-1] = biases[number_of_levels] + np.matmul(weights[number_of_levels],h[number_of_levels-1])
        pred_y = output(a[number_of_levels-1])
        return a,h, pred_y


    def gradient_descent():
        a,h, pred_y = forward_propagate()
        delthet = backward_propagate(a,h, pred_y)
        thet += delthet

    def fit(dataset):
        for x,y in dataset:
            loss = forward(x,y)
            delthet = backward(loss)
            thet += learn_rate*delthet
    